In [97]:
import os,sys

os.getcwd()

'/home/fin/tmp/fis-ski'

In [166]:
import unicodecsv

def read_lines(reader):
    headers = reader.next()
    for x in reader:
        yield dict(zip(headers,x))

all_ergebnisse = []
errorcount=0
errors = []

import re
numbersonly = re.compile('=([0-9]+)')

all_races = {}

with open('fin_output.csv') as ergebnisse, open('subpages.csv') as subpages:
    er = read_lines(unicodecsv.reader(ergebnisse))
     
    sr = unicodecsv.reader(subpages)
    
    all_races = dict((numbersonly.findall(x['url'])[0],x,) for x in read_lines(sr))
    for r in all_races.values():
        ds = r['date'].split('.')
        r['year'] = ds[2]
        r['month'] = ds[1]
        r['day'] = ds[0]

    
    for e in er:
        try:
            e['race'] = all_races[numbersonly.findall(e['source_url'])[0]]
            all_ergebnisse.append(e)
        except:
            errorcount+=1
            errors.append(e)

In [167]:
def sortgroup(xs, keyfun):
    import itertools
    return itertools.groupby(sorted(xs,key=keyfun),key=keyfun)

by_race = dict([(x,
                 list(sorted(
                    [z for z in y if z['Rank'] and z['Rank'].isdigit()]
                    , key=lambda e: int(e['Rank'],10))),
                )
                for (x,y,) in sortgroup(all_ergebnisse, lambda x: x['source_url'])
               ])

In [169]:
pointreplaced = 0

for raceurl,finishers in by_race.iteritems():

    last = None
    for f in finishers:
        if f['Total Time'].strip():
            if last:
                if ':' in last['Total Time'] and not ':' in f['Total Time']:
                    pointreplaced+=1
                    f['Total Time'] = f['Total Time'].replace('.',':',1)
            if ':' in f['Total Time']:
                minutes,rest = f['Total Time'].split(':',1)
            else:
                minutes = '0'
                rest = f['Total Time']
            
            seconds,nths = re.split('[\.,]',rest)

            minutes = int(minutes,10)
            seconds = int(seconds,10)
            nths_i = int(nths,10)
            nths_f = nths_i/(pow(10.0,len(nths)))
            finalseconds = minutes*60 + seconds + nths_f

            f['mytime'] = finalseconds

            f['diff'] = f['mytime']-finishers[0]['mytime']
            if last and hasattr(last,'mytime'):
                if f['diff']<last['diff']:
                    print 'time travel?'
                    print f['Total Time'],f['mytime'],last['Total Time'],last['mytime'],f['source_url']
                    print seconds
                    print nths, len(nths), nths_i, nths_f
    
        last = f

In [329]:
with open('ergebnisse_w_diffs.csv','w') as f:
    cw = unicodecsv.writer(f)
    f = by_race.values()[0][0]
    fk = f.keys()
    rk = f['race'].keys()
    headers = fk+rk
    cw.writerow(headers)
    for y in by_race.values():
        for x in y:
            if x['Total Time'].strip():
                cw.writerow(
                    [x[k] for k in fk] + [x['race'][k] for k in rk])

In [170]:
import itertools
all_diffs = [x for x in itertools.chain(*by_race.values()) if x.get('diff',None)]

In [252]:
import collections

MAX_SECONDS = 4

all_fitting = {k:[{'Rank': x['Rank'],
                   'Name': x['Name'],
                   'Time': x['Total Time'],
                   'diff': x['diff'],
                   'time': x['mytime']
                  } for x in vv if x.get('diff',None) is not None and x['diff']<=MAX_SECONDS] for k,vv in by_race.iteritems()
               if vv}

In [253]:
result = []
for k,v in all_fitting.iteritems():
    if not v:
        continue
    r = {}
    r.update(by_race[k][0]['race'])
    r['ranks'] = v
    result.append(r)

In [254]:
last_season_races = [x for x in result
 if ('2015'==x['year'] and int(x['month'],10)>6)
 or (x['year']=='2016' and int(x['month'],10)<=6)]

In [255]:
for race in last_season_races:
    tmp = [x['diff'] for x in race['ranks'] if int(x['Rank'],10)<=5]
    race['knappheit'] = sum(tmp)/len(tmp)
    race['diffzweiter'] = race['ranks'][1]['diff']
    race['finishersinnsecs'] = len([x for x in race['ranks'] if x['diff']<=MAX_SECONDS])

In [291]:
by_disc = dict((x,list(y),) for x,y in sortgroup(last_season_races,
                                                 lambda x: '%s (%s)' % (x['disziplin'],x['gender'],)))

In [326]:
def mean(numbers):
    return sum(numbers)/(1.0*len(numbers))

MAX_RANK = 15

with open('out.csv','w') as f:
    cw = unicodecsv.writer(f)
    
    cw.writerow(['Disziplin','Rennen']+
               list(itertools.chain.from_iterable([['Platz %s'%i, 'Name %s'%i] for i in xrange(1,MAX_RANK+1)])))

    for disc,discraces in by_disc.iteritems():
        closest = sorted(discraces, key=lambda x:x['diffzweiter'])[0]
        cw.writerow([disc,'closest (%s / %s)'%(closest['ort'],closest['date'],)]+
                    list(itertools.chain.from_iterable([
                     [round(x['diff'],2),x['Name']]
                      for x in closest['ranks'][:MAX_RANK+1]
                     ]
                   )
                    ))
        average = [mean([x['diff'] for x in reduce(lambda x,y: x+y, [z['ranks'] for z in discraces],[]) if int(x['Rank'],10)==y]) for y in xrange(1,MAX_RANK+1)]
        cw.writerow([disc,'mean']+
                    list(itertools.chain.from_iterable([
                        [round(x,2),'%d. Platz' % (i+1)] for i,x in enumerate(average)
                    ]))
           )

In [325]:
closest

{u'category': u'WC',
 u'codex': u'5175',
 u'date': u'16.03.2016',
 'day': u'16',
 'diffzweiter': 0.12999999999999545,
 u'disziplin': u'Downhill',
 'finishersinnsecs': 21,
 u'gender': u'L',
 'knappheit': 0.25799999999999274,
 u'kommentar': u'',
 u'land': u'SUI',
 'month': u'03',
 u'ort': u'St. Moritz',
 'ranks': [{'Name': u'PUCHNER Mirjam',
   'Rank': u'1',
   'Time': u'1:40.93',
   'diff': 0.0,
   'time': 100.93},
  {'Name': u'SUTER Fabienne',
   'Rank': u'2',
   'Time': u'1:41.06',
   'diff': 0.12999999999999545,
   'time': 101.06},
  {'Name': u'CURTONI Elena',
   'Rank': u'3',
   'Time': u'1:41.10',
   'diff': 0.1699999999999875,
   'time': 101.1},
  {'Name': u'GOERGL Elisabeth',
   'Rank': u'4',
   'Time': u'1:41.38',
   'diff': 0.44999999999998863,
   'time': 101.38},
  {'Name': u'SCHNARF Johanna',
   'Rank': u'5',
   'Time': u'1:41.47',
   'diff': 0.539999999999992,
   'time': 101.47},
  {'Name': u'STUFFER Verena',
   'Rank': u'6',
   'Time': u'1:41.48',
   'diff': 0.5499999999999

In [61]:
races_with_time = set([x['raceurl'] for x in all_ergebnisse if x['rang'].strip() and x['time'].strip()])
races_witout_time = set([x['raceurl'] for x in all_ergebnisse if x['rang'].strip() and not x['time'].strip()])

In [63]:
len(races_with_time & races_witout_time)

45

In [289]:
auswertungen = {
 '1': lambda x: x['Rank']=='1',
 '1 at': lambda x: x['Rank']=='1' and x['Nation']=='AUT',
 '2': lambda x: x['Rank']=='2',
 '2 at': lambda x: x['Rank']=='2' and x['Nation']=='AUT',
 '3': lambda x: x['Rank']=='3',
 '3 at': lambda x: x['Rank']=='3' and x['Nation']=='AUT',
}

all_by_disziplin = {k: list(v) for k,v in sortgroup(all_ergebnisse,
                                                    lambda x: 
                                                    '%s (%s)' % (x['race']['disziplin'],x['race']['gender'],)
                                                   )}

auswertungen_by_disziplin = {
    k: {aname: len(list(x for x in v if aprop(x)))
        for aname, aprop
        in auswertungen.iteritems()
       }
    for k,v in all_by_disziplin.iteritems()
}

In [294]:
with open('allraces.csv','w') as f:
    cw = unicodecsv.writer(f)
    cw.writerow(['Kategorie','Platz', 'Wert'])
    for k,v in auswertungen_by_disziplin.iteritems():
        for k2, v2 in v.iteritems():
            cw.writerow([k,k2,v2])

In [286]:
all_ergebnisse[0]

{u'Bib': u'',
 u'Diff.': u'&nbsp',
 u'FIS Code': u'55198',
 u'FIS Points': u'-4.00',
 u'Name': u'DORFMEISTER Michaela',
 u'Nation': u'AUT',
 u'Rank': u'1',
 u'Run 1': u'1:07.95',
 u'Run 2': u'1:11.93',
 u'Total Time': u'2:19.88',
 u'Year': u'1973',
 'diff': 0.0,
 'mytime': 139.88,
 'race': {u'category': u'WC',
  u'codex': u'5127',
  u'date': u'09.12.2000',
  'day': u'09',
  u'disziplin': u'Giant Slalom',
  u'gender': u'L',
  u'kommentar': u'Val d Is\xe8re06.12',
  u'land': u'ITA',
  'month': u'12',
  u'ort': u'Sestriere',
  u'url': u'http://data.fis-ski.com/dynamic/results.html?sector=AL&raceid=13100',
  'year': u'2000'},
 u'source_url': u'http://data.fis-ski.com/dynamic/results.html?sector=AL&raceid=13100'}